In [1]:
# Dependencies
# For extracting
import requests
from config_env import api_key, g_key, w_key
import time
# For transformation
import json

#for scraping Guardian article
from bs4 import BeautifulSoup as bs

import pymongo
from pymongo import MongoClient
import os
import urllib.parse
from config_env import username, password
dbstring = "@ds163764.mlab.com:63764/heroku_pptb0bm8"


In [2]:
uri = f'mongodb://{username}{password}{dbstring}'
client = MongoClient(uri)

In [3]:
db = client.heroku_pptb0bm8

In [4]:
def cloud_insert(output_list):
    uri = f'mongodb://{username}{password}{dbstring}'
    client = MongoClient(uri)
    db = client.heroku_pptb0bm8
    try:
        db.news_api.insert_many(output_list)
    except TypeError:
        print("Error inserting")
        

In [ ]:
def nyt_api(begin_date, end_date, query, pages):
    from config_env import api_key
    base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
    # Store a search term
    param = 'fq=news_desk:(Foreign)&fq=source:(The New York Times)'
    # Search for articles published between a begin and end date
    page = '&page='
    query_url = f"{base_url}{param}&api-key={api_key}&q={query}&facet_field=snippet&begin_date={begin_date}&end_date={end_date}{page}"
    
    output_list = []
# Retrieve articles

    for n in range(0, pages):
        url=query_url + str(n)
        try:
            articles = requests.get(url).json()
            print(f"Getting data from page {n} from NY Times query...")
            articles_list = [article for article in articles["response"]["docs"]]

            for item in articles_list:
                list_dict = {}
                list_dict["url"] = item["web_url"]
                list_dict["headline"] = item["headline"]["main"]
                list_dict["pub_date"] = item["pub_date"]
                list_dict["source"] = "NYT"
                output_list.append(list_dict)

            time.sleep(6)
        except KeyError:
            print(f"Error with {n}")
            pass
    cloud_insert(output_list)

In [ ]:
nyt_api('20190222', '20190223', 'Venezuela', 2)

In [ ]:
def guardian_api(begin_date, query, pages):
    # Store a search term

    url = f"https://content.guardianapis.com/search?section=world&page-size=100&from-date={begin_date}&q={query}&api-key={g_key}&page="
    output_list = []
    # For the Guardian API
    for i in range(1, pages):
        query_url = url + str(i)
        print(f'Getting data from page {pages} of Guardian query...')
        try:
            articles = requests.get(query_url).json()
            if articles['response']['status'] == 'ok': 
                articles_list = [article for article in articles["response"]["results"]]
                for item in articles_list:
                    if item["type"] == 'article':
                        list_dict = {}
                        list_dict["url"] = item["webUrl"]
                        list_dict["headline"] = item["webTitle"]
                        list_dict["pub_date"] = item["webPublicationDate"]
                        list_dict["source"] = "Guardian"
                        output_list.append(list_dict)
        except KeyError:
            pass
    cloud_insert(output_list)

In [ ]:
def article_text(collection):
    nyt = []
    guard = []
    for item in collection:
        if item['source'] == 'NYT': # If this is from the NY Times do this...
            url = item['url']
            nyt_dict = {}
            print(f"Getting article text for {item['url']}...") 
            # Use Requests to get the HTML, verify False gets around https verification
            try:
                response = requests.get(url, verify=False)
                soup = bs(response.text, 'lxml')
                # Class specifies the body of the article
                body_copy = soup.find('div', class_='css-53u6y8')
            except AttributeError:
                pass
            nyt_dict['body_copy'] = str(body_copy)
            # Insert this back into the database matching the URL
            db.news.update_one({'url':url}, {'$set': nyt_dict})
            time.sleep(1)
        
        elif item['source'] == 'Guardian':
            g_dict = {}
            url = item['url']
            print(f"Getting article text for {item['url']}...") 
            try:
                response = requests.get(url)
                soup = bs(response.text, 'html.parser')
                results = soup.find('div', class_="content__article-body")
            except AttributeError:
                pass
            text = results.find_all('p')
            g_dict['body_copy'] = str(text)
            db.news.update_one({'url':url}, {'$set': g_dict})


In [ ]:
guardian_api('2019-02-23', 'Venezuela',2)

In [ ]:
# Go in and fill in any empty body text
news = db.news.find({'body_copy': { '$exists': False}})
article_text(news)

In [5]:
def news_api(query, w_key, begin_date, end_date, pages):
    #Dates are MM-DD-YYYY
    url = f"https://newsapi.org/v2/everything?q={query}&from={begin_date}&to=\
    {end_date}&language=en&pageSize=100&apiKey={w_key}&page="
    
    output_list = []
    # For the News API
    for i in range(1, pages):
        query_url = url + str(i)
        print(f'Getting data from page {query_url} of News API query...')
        try:
            articles = requests.get(query_url).json()
            if articles['status'] == 'ok': 
                articles_list = [article for article in articles["articles"]]
                for item in articles_list:
                    list_dict = {}
                    list_dict["url"] = item["url"]
                    list_dict["headline"] = item["title"]
                    list_dict["pub_date"] = item["publishedAt"]
                    list_dict['body_copy'] = item['content']
                    list_dict["source"] = item['source']['name']
                    output_list.append(list_dict)
        except KeyError:
            pass
        try:
            cloud_insert(output_list)
        except TypeError:
            pass
    

In [7]:
news_api('Venezuela', w_key, '01-23-2019', '02-23-2019', 3)

Getting data from page https://newsapi.org/v2/everything?q=Venezuela&from=01-23-2019&to=    02-23-2019&language=en&pageSize=100&apiKey=fc19f7c59a6d4a95b295bab5e56f21b3&page=1 of News API query...
Getting data from page https://newsapi.org/v2/everything?q=Venezuela&from=01-23-2019&to=    02-23-2019&language=en&pageSize=100&apiKey=fc19f7c59a6d4a95b295bab5e56f21b3&page=2 of News API query...


BulkWriteError: batch op errors occurred